In [57]:
from easydict import EasyDict as edict
import torch
import os
import gzip

args = edict()
# arguments
args.mode = 'test'
args.dataset = "Cell_Phones_and_Accessories"
assert args.mode in ['train', 'test']

# settings
args.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args.save_dir = args.dataset.split('_')[0]

# will be use
args.vae_model_path = f"./{args.save_dir}/model/vae_model.pth"
args.rec_model_path = f"./{args.save_dir}/model/rec_model.pth"

args.inter_data_dir = f"./{args.save_dir}/{args.mode}/"

# KG related file
args.tmp_dir = args.dataset
args.kg_users_file = '{}/kg_users_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_items_file = '{}/kg_items_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_entities_file = '{}/kg_entities_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_relations_file = '{}/kg_relations_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_train_triples_file = '{}/kg_train_triples_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_train_gt_file = '{}/kg_train_gt_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_other_triples_file = '{}/kg_other_triples_{}.txt'.format(args.tmp_dir, args.dataset)

args.kg_test_triples_file = '{}/kg_test_triples_{}.txt'.format(args.tmp_dir, args.dataset)
args.kg_test_candidates_file = '{}/kg_test_candidates_{}.txt.gz'.format(args.tmp_dir, args.dataset)

args.transE_embedding_file = '{}/openke/transe_epoch_100_lr_0.002.ckpt'.format(args.tmp_dir, args.dataset)


In [72]:
from os import defpath
#from utils import args
import numpy as np
from collections import defaultdict
#import torch
from tqdm import tqdm

#%%
class KnowledgeGraph():
    def __init__(self, kg_file, purchase_file):
        self.kg_file = kg_file
        self.purchase_file = purchase_file
        self.G = defaultdict(list)
        self.uip = defaultdict(list) # user purchased item
        self.ui = defaultdict(list)
        self._load_kg()


    def _load_kg(self):
        with open(self.kg_file, 'r') as f:
            for line in f:
                k, v, r = map(int, line.rstrip().split('\t'))
                self.G[k].append((v,r))
                self.G[v].append((k,r)) # reverse edge
        with open(self.purchase_file, 'r') as f:
            for line in f:
                k, v, r = map(int, line.rstrip().split('\t'))
                self.uip[k].append((v,r))
                self.ui[k].append(v)
                self.G[v].append((k,r)) # reverse edge
        # remove repeated edges
        for k in self.G:
            self.G[k] = list(set(self.G[k]))
        for k in self.uip:
            self.uip[k] = list(set(self.uip[k]))
        for k in self.ui:
            self.ui[k] = list(set(self.ui[k]))

    def get_k_hop_path(self, u, k, mode, items=None):
        path = []
        if mode == 'train':
            to_visit = [self.G[u].copy()]
            items = set(self.ui[u])
        else:
            assert items is not None
            to_visit = [self.G[u].copy()+self.uip[u].copy()]
            items = set(items)
        current_path = [(u, -1)]
        d = 1
        while True:
            if d == k:
                for u,i in to_visit[-1]:
                    if u in items:
                        path.append(current_path+[(u,i)])
                current_path.pop()
                to_visit.pop()
                d -= 1
            try:
                u, p = to_visit[-1].pop()
                # if u, p == 149744, 32:
                #     print(current_path)
            except IndexError:
                d-=1
                try:
                    to_visit.pop()
                except IndexError:
                    break
                current_path.pop()
                continue
            current_path.append((u, p))
            # to_visit.append(self.G[u].copy()+self.uip[u].copy())
            # to_visit.append(set(list(zip(*(self.G[u].copy()+self.uip[u].copy())))[0]) - set(list(zip(*current_path)))[0])
            to_visit.append([])
            for uu, pp in set(self.G[u].copy()+self.uip[u].copy()):
                if uu not in set(list(zip(*current_path))[0]):
                    to_visit[-1].append((uu, pp))

            d += 1
        return path

In [15]:
KG.ui[1]

[84836, 106439, 81811, 68021, 79899, 106140]

In [73]:
KG = KnowledgeGraph(args.kg_other_triples_file, args.kg_train_triples_file)

In [62]:
test_cand = []
with gzip.open(args.kg_test_candidates_file, 'rt') as f:
    for line in f:
        test_cand.append(list(map(int, line.rstrip().split('\t'))))

In [71]:
def f(i,k):
    b = []
    u = test_cand[i::k]
    for uu in u:
        a = KG.get_k_hop_path(uu[0],3,'test',uu[1:])
        if a:
            b.append(a)
    return b

In [1]:
len(test_cand[0])

NameError: name 'test_cand' is not defined

In [82]:
import multiprocessing
multiprocessing.cpu_count()

8

In [7]:
from threading import Thread
import pickle

In [8]:
def f(i,k):
    b = []
    u = list(KG.ui.keys())[i::k]
    for uu in u:
        a = KG.get_k_hop_path(uu,3)
        if a:
            b.append(a)
    return b

In [74]:
from time import time
from  multiprocessing import Process,Pool
k=8
t = []
result = []
start = time()
pool = Pool(k)
for i in range(k):
    #t.append(Thread(target=f, args=(list(KG.ui.keys())[i::k])))
    t.append(pool.apply_async(func=f, args=(i,k)))
    #t[-1].setDaemon(True)

pool.close()
pool.join()
for tt in t:
    result += tt.get()
print(time()-start)

14.276602268218994


In [17]:
12448/60/60

3.457777777777778

In [ ]:
from time import time
k=6
list(KG.ui.keys())[::k]
t = []
result = []
start = time()
for i in range(k):
    #t.append(Thread(target=f, args=(list(KG.ui.keys())[i::k])))
    t.append(Thread(target=f, args=(i,)))
    #t[-1].setDaemon(True)
    t[-1].start()

# for tt in t:
#     tt.join()
print(time()-start)
# for tt in t:
#     result = result + tt.getResult()

In [80]:
result[0]

[[(0, -1), (79580, 0), (139373, 8), (76747, 8)],
 [(0, -1), (79580, 0), (139373, 8), (65026, 8)],
 [(0, -1), (79580, 0), (139373, 8), (86543, 8)],
 [(0, -1), (79580, 0), (139373, 8), (80827, 8)],
 [(0, -1), (79580, 0), (139373, 8), (100430, 8)],
 [(0, -1), (79580, 0), (139373, 8), (72603, 8)],
 [(0, -1), (79580, 0), (139373, 8), (75930, 8)],
 [(0, -1), (79580, 0), (139373, 8), (80376, 8)],
 [(0, -1), (79580, 0), (139373, 8), (93429, 8)],
 [(0, -1), (79580, 0), (139373, 8), (86408, 8)],
 [(0, -1), (79580, 0), (139373, 8), (83360, 8)],
 [(0, -1), (79580, 0), (139373, 8), (100730, 8)],
 [(0, -1), (79580, 0), (139373, 8), (64014, 8)],
 [(0, -1), (79580, 0), (139373, 8), (80398, 8)],
 [(0, -1), (79580, 0), (139373, 8), (79443, 8)],
 [(0, -1), (79580, 0), (139373, 8), (76152, 8)],
 [(0, -1), (79580, 0), (139373, 8), (91389, 8)],
 [(0, -1), (79580, 0), (139373, 8), (71092, 8)],
 [(0, -1), (79580, 0), (139373, 8), (87526, 8)],
 [(0, -1), (79580, 0), (139373, 8), (64568, 8)],
 [(0, -1), (79580,

In [76]:
len(result)

10

In [19]:
len(result[135])

16

In [47]:
result[0]

[[(0, -1), (114104, 1), (75595, 2), (71092, 6)],
 [(0, -1), (114104, 1), (100895, 2), (71092, 6)],
 [(0, -1), (114104, 1), (36330, 1), (75711, 0)],
 [(0, -1), (114104, 1), (94941, 2), (71092, 6)]]

In [32]:
if not os.path.exists(args.inter_data_dir):
    os.makedirs(args.inter_data_dir)
with open(args.inter_data_dir + 'all_path.txt','wb') as fp:
    pickle.dump(result, fp)

In [ ]:
with open(args.inter_data_path + 'all_path.txt','rb') as fp:
    result = pickle.load(fp)

In [33]:
import torch
import itertools

In [34]:
ckpt = torch.load(args.transE_embedding_file, map_location=torch.device('cpu'))
embeds = ckpt['ent_embeddings.weight'].cpu().numpy()
rels = ckpt['rel_embeddings.weight'].cpu().numpy()
print(type(embeds), embeds.shape)
print(type(rels), rels.shape)

<class 'numpy.ndarray'> (169331, 100)
<class 'numpy.ndarray'> (45, 100)


In [35]:
chain = itertools.chain(*result)
re = list(chain)
re = np.array(re, dtype=int)
re.shape

(2137377, 4, 2)

In [36]:
re[0]

array([[     0,     -1],
       [114104,      1],
       [ 75595,      2],
       [ 71092,      6]])

In [37]:
nodes = re[:,:,0]
paths = re[:,:,1]

In [39]:
dist = embeds[nodes[:,1:]] - embeds[nodes[:,:-1]] - rels[paths[:,1:]] # a->b  a+r=b
dist = np.sum(np.linalg.norm(dist,ord=1, axis=-1), axis=-1) ###L1

In [40]:
dist.shape

(2137377,)

In [41]:
import numpy_indexed as npi
ui, idx = npi.group_by(re[:,[0,3],0]).argmin(dist)

In [44]:
opti_paths = re[idx]

In [42]:
def represent_avg(v):
    return np.average(v, axis=-2)

In [45]:
opti_paths.shape

(96134, 4, 2)

In [46]:
opti_paths[0]

array([[     0,     -1],
       [114104,      1],
       [100895,      2],
       [ 71092,      6]])

In [48]:
np.save(args.inter_data_dir+'opti_paths.npy',opti_paths)

In [49]:
nodes_emb = embeds[opti_paths[:,:,0]]
edges_emb = rels[opti_paths[:,1:,1]]

In [50]:
opti_paths_emb = np.concatenate([nodes_emb, edges_emb],axis=1)

In [51]:
rep = represent_avg(opti_paths_emb)

In [52]:
rep.shape

(96134, 100)

In [53]:
np.save(args.inter_data_dir+'opti_path_rep.npy',rep)